## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import torch
import warnings
warnings.filterwarnings('ignore')
from itertools import product

## Load Transcript Data

Load the transcripts generated by Whisper from the previous notebook.

In [ ]:
def load_transcripts(folder_path, label):
    """
    Load all transcript files from a folder.
    
    Args:
        folder_path: Path to folder containing transcript .txt files
        label: 0 for bad, 1 for good
    
    Returns:
        List of dicts with 'text' and 'label' keys
    """
    folder = Path(folder_path)
    
    if not folder.exists():
        print(f"⚠️  Folder not found: {folder_path}")
        return []
    
    data = []
    txt_files = sorted(folder.glob("*.txt"))
    
    for txt_file in txt_files:
        try:
            text = txt_file.read_text(encoding='utf-8').strip()
            if text:  # Skip empty files
                data.append({
                    'text': text,
                    'label': label
                })
        except Exception as e:
            print(f"❌ Error reading {txt_file.name}: {e}")
    
    return data

# Load good answers (label=1)
good_data = load_transcripts("../data/good_answer_transcripts", label=1)
print(f"✅ Loaded {len(good_data)} good answers")

# Load bad answers (label=0)
bad_data = load_transcripts("../data/bad_answer_transcripts", label=0)
print(f"❌ Loaded {len(bad_data)} bad answers")

# Combine datasets
all_data = good_data + bad_data
print(f"\n📊 Total samples: {len(all_data)}")

✅ Loaded 33 good answers
❌ Loaded 13 bad answers

📊 Total samples: 46


In [ ]:
# Convert to DataFrame
df = pd.DataFrame(all_data)

display(df.head())

,text,label
0,"זה בעצם לא יחוג, צריך להבין קודם בעזחוג את המת...",1
1,"אוקיי, אתה רוצה להתקבל לתואר הראשון בהנדסה ביר...",1
2,מהבחינת את דמרישום אז בונראם בעצם יש כאלה שפתו...,1
3,"חלק מהטיימה, אוניברסיטאים לטור שון הם בעצם ידע...",1
4,בחלק מהאכוגים בפקולטות למדער הוא אחר או בפקולט...,1


## Define Evaluation Metrics

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

def compute_metrics(pred):
    labels = pred.label_ids
    logits = pred.predictions

    preds = logits.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary", zero_division=0
    )
    acc = accuracy_score(labels, preds)

    # AUC needs probabilities for class 1
    probs_pos = np.exp(logits) / np.exp(logits).sum(axis=1, keepdims=True)
    auc = roc_auc_score(labels, probs_pos[:, 1])

    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall, "auc": auc}

## Check Class Distribution and Setup Hyperparameter Grid

In [ ]:
# Hyperparameter search space
param_grid = {
    'learning_rate': [1e-5, 2e-5, 3e-5],
    'num_train_epochs': [15, 20, 25],
    'per_device_train_batch_size': [4, 8],
    'weight_decay': [0.01, 0.02, 0.05],
    'warmup_ratio': [0.0, 0.1, 0.2],
}

# Model to use
model_names = [
    "onlplab/alephbert-base",
]

# Random search configuration
total_combinations = int(np.prod([len(v) for v in param_grid.values()]))
n_random_samples = 20  # Try 20 random combinations instead of all 162
n_folds = 5

print("Hyperparameter Random Search Configuration")
print("=" * 50)
print(f"Model: {model_names[0]}")
print(f"Total possible combinations: {total_combinations}")
print(f"Random samples to try: {n_random_samples}")
print(f"CV folds per sample: {n_folds}")
print(f"Total training runs: {n_random_samples * n_folds}")
print(f"Coverage: {n_random_samples/total_combinations*100:.1f}% of search space")
print("\nParameter search space:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")

Hyperparameter Random Search Configuration
Model: onlplab/alephbert-base
Total possible combinations: 162
Random samples to try: 20
CV folds per sample: 5
Total training runs: 100
Coverage: 12.3% of search space

Parameter search space:
  learning_rate: [1e-05, 2e-05, 3e-05]
  num_train_epochs: [15, 20, 25]
  per_device_train_batch_size: [4, 8]
  weight_decay: [0.01, 0.02, 0.05]
  warmup_ratio: [0.0, 0.1, 0.2]


## Cross-Validation Training Function

Function to train model with k-fold cross-validation and return mean AUC.

In [ ]:
def train_with_cv(model_name, params, df, n_splits=5):
    """
    Train model with stratified k-fold cross-validation.
    
    Args:
        model_name: HuggingFace model name
        params: Dictionary of hyperparameters
        df: DataFrame with 'text' and 'label' columns
        n_splits: Number of CV folds
    
    Returns:
        mean_auc: Average AUC across folds
        std_auc: Standard deviation of AUC
        fold_aucs: List of AUC scores for each fold
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_aucs = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(df['text'], df['label'])):
        print(f"  Fold {fold + 1}/{n_splits}", end=" ")
        
        # Split data
        train_texts = df.iloc[train_idx]['text'].tolist()
        train_labels = df.iloc[train_idx]['label'].tolist()
        val_texts = df.iloc[val_idx]['text'].tolist()
        val_labels = df.iloc[val_idx]['label'].tolist()
        
        # Load model and tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=2
        )
        
        # Create datasets
        train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
        val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels})
        
        # Tokenize
        def tokenize_function(examples):
            return tokenizer(examples['text'], truncation=True, max_length=512)
        
        train_dataset = train_dataset.map(tokenize_function, batched=True)
        val_dataset = val_dataset.map(tokenize_function, batched=True)
        
        train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
        val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
        
        # Training arguments
        training_args = TrainingArguments(
            output_dir=f"../models/cv_temp_fold_{fold}",
            eval_strategy="epoch",
            save_strategy="no",  # Don't save during CV
            learning_rate=params['learning_rate'],
            lr_scheduler_type="linear",
            warmup_ratio=params['warmup_ratio'],
            per_device_train_batch_size=params['per_device_train_batch_size'],
            per_device_eval_batch_size=16,
            gradient_accumulation_steps=4,
            num_train_epochs=params['num_train_epochs'],
            weight_decay=params['weight_decay'],
            load_best_model_at_end=False,
            metric_for_best_model="auc",
            greater_is_better=True,
            logging_steps=1000,  # Reduce logging during CV
            report_to="none",
            seed=42 + fold,
        )
        
        # Data collator
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
        
        # Trainer with early stopping
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        )
        
        # Train
        trainer.train()
        
        # Evaluate
        eval_result = trainer.evaluate()
        fold_auc = eval_result['eval_auc']
        fold_aucs.append(fold_auc)
        
        print(f"→ AUC: {fold_auc:.4f}")
        
        # Clean up memory
        del model, trainer, tokenizer
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    mean_auc = np.mean(fold_aucs)
    std_auc = np.std(fold_aucs)
    
    return mean_auc, std_auc, fold_aucs

print("✅ Cross-validation function defined")

✅ Cross-validation function defined


## Run Random Hyperparameter Search with Cross-Validation

Randomly sample parameter combinations to efficiently explore the search space.

In [ ]:
# Run random search with cross-validation
results = []
best_auc = 0
best_config = None

print("=" * 60)
print("STARTING RANDOM HYPERPARAMETER SEARCH WITH CV")
print("=" * 60)
print("Objective: Maximize AUC\n")

for model_idx, model_name in enumerate(model_names, 1):
    print(f"\n{'='*60}")
    print(f"MODEL {model_idx}/{len(model_names)}: {model_name}")
    print(f"{'='*60}\n")
    
    # Generate random parameter combinations
    np.random.seed(42)  # For reproducibility
    tested_configs = set()  # Avoid duplicates
    param_combinations = []
    
    while len(param_combinations) < n_random_samples:
        # Randomly sample one value from each parameter
        random_params = {
            param: np.random.choice(values)
            for param, values in param_grid.items()
        }
        
        # Convert numpy types to native Python types
        random_params = {
            k: int(v) if isinstance(v, (np.integer, np.int64, np.int32)) else 
               float(v) if isinstance(v, (np.floating, np.float64, np.float32)) else v
            for k, v in random_params.items()
        }
        
        # Convert to hashable format to check for duplicates
        config_hash = tuple(sorted(random_params.items()))
        if config_hash not in tested_configs:
            tested_configs.add(config_hash)
            param_combinations.append(random_params)
    
    for i, params in enumerate(param_combinations, 1):
        print(f"[Config {i}/{n_random_samples}] ", end="")
        print(f"LR={params['learning_rate']}, Epochs={params['num_train_epochs']}, ", end="")
        print(f"BS={params['per_device_train_batch_size']}, WD={params['weight_decay']}, ", end="")
        print(f"Warmup={params['warmup_ratio']}")
        
        try:
            mean_auc, std_auc, fold_aucs = train_with_cv(model_name, params, df, n_splits=5)
            
            result = {
                'model_name': model_name,
                'params': params,
                'mean_auc': mean_auc,
                'std_auc': std_auc,
                'fold_aucs': fold_aucs
            }
            results.append(result)
            
            print(f"  ➜ Mean AUC: {mean_auc:.4f} ± {std_auc:.4f}")
            
            # Track best configuration
            if mean_auc > best_auc:
                best_auc = mean_auc
                best_config = result
                print(f"  🏆 NEW BEST AUC!")
                
        except Exception as e:
            print(f"  ❌ Error: {e}")
            continue

print("\n" + "=" * 60)
print("RANDOM SEARCH COMPLETE!")
print("=" * 60)
print(f"Tested {len(results)}/{n_random_samples} configurations")
print(f"Best AUC: {best_auc:.4f}")
print(f"Best model: {best_config['model_name']}")
print("\nBest parameters:")
for k, v in best_config['params'].items():
    print(f"  {k}: {v}")

STARTING RANDOM HYPERPARAMETER SEARCH WITH CV
Objective: Maximize AUC


MODEL 1/1: onlplab/alephbert-base

[Config 1/20] LR=3e-05, Epochs=15, BS=4, WD=0.05, Warmup=0.0
  Fold 1/5 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Auc
1,No log,0.663864,0.700000,0.823529,0.700000,1.000000,0.238095
2,No log,0.646697,0.700000,0.823529,0.700000,1.000000,0.238095
3,No log,0.629840,0.700000,0.823529,0.700000,1.000000,0.238095
4,No log,0.623983,0.700000,0.823529,0.700000,1.000000,0.619048
5,No log,0.626557,0.700000,0.823529,0.700000,1.000000,0.714286
6,No log,0.621336,0.700000,0.823529,0.700000,1.000000,0.714286
7,No log,0.606873,0.700000,0.823529,0.700000,1.000000,0.761905
8,No log,0.588356,0.700000,0.823529,0.700000,1.000000,0.809524
9,No log,0.573347,0.700000,0.823529,0.700000,1.000000,0.761905
10,No log,0.562868,0.700000,0.823529,0.700000,1.000000,0.761905


## Prepare Data and Train Final Model with Best Configuration

Now split data 80/20 and train final model using the best hyperparameters found.

In [ ]:
# Load best model and tokenizer
model_name = best_config['model_name']
best_params = best_config['params']

# Ensure all params are native Python types (not numpy)
best_params = {
    k: int(v) if isinstance(v, (np.integer, np.int64, np.int32)) else 
       float(v) if isinstance(v, (np.floating, np.float64, np.float32)) else v
    for k, v in best_params.items()
}

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

print(f"Loaded {model_name}")
print(f"\nBest parameters from CV (AUC: {best_config['mean_auc']:.4f} ± {best_config['std_auc']:.4f}):")
for k, v in best_params.items():
    print(f"  {k}: {v}")

# Split data (80/20)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

print(f"\nTrain set: {len(train_texts)} samples")
print(f"Validation set: {len(val_texts)} samples")

# Create HuggingFace datasets
train_dataset = Dataset.from_dict({
    'text': train_texts,
    'label': train_labels
})

val_dataset = Dataset.from_dict({
    'text': val_texts,
    'label': val_labels
})

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=512)

# Apply tokenization
print("\nTokenizing datasets...")
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

print("✅ Tokenization complete!")

# Training arguments using best parameters
training_args = TrainingArguments(
    output_dir="../models/customer-support-classifier",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_params['learning_rate'],
    lr_scheduler_type="linear",
    warmup_ratio=best_params['warmup_ratio'],
    per_device_train_batch_size=best_params['per_device_train_batch_size'],
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    num_train_epochs=best_params['num_train_epochs'],
    weight_decay=best_params['weight_decay'],
    load_best_model_at_end=True,
    metric_for_best_model="auc",
    greater_is_better=True,
    logging_dir="../models/logs",
    logging_steps=10,
    save_total_limit=2,
    report_to="none",
)

print("\n✅ Training configuration ready!")

## Initialize Trainer with Early Stopping

In [ ]:
# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize Trainer with early stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print("✅ Trainer initialized with early stopping!")

## Train Model

This may take several minutes depending on hardware.

In [ ]:
print("Starting training...\n")
train_result = trainer.train()

print("\n" + "="*50)
print("TRAINING COMPLETE!")
print("="*50)
print(f"Train loss: {train_result.training_loss:.4f}")
print(f"Training time: {train_result.metrics['train_runtime']:.2f}s")

## Evaluate Model

In [ ]:
print("Evaluating on validation set...\n")
eval_result = trainer.evaluate()

print("="*50)
print("EVALUATION RESULTS")
print("="*50)
print(f"Accuracy:  {eval_result['eval_accuracy']:.4f}")
print(f"Precision: {eval_result['eval_precision']:.4f}")
print(f"Recall:    {eval_result['eval_recall']:.4f}")
print(f"F1 Score:  {eval_result['eval_f1']:.4f}")
print(f"Loss:      {eval_result['eval_loss']:.4f}")

## Save Fine-tuned Model

In [ ]:
# Save the best model
output_dir = "../models/customer-support-classifier-final"
Path(output_dir).mkdir(parents=True, exist_ok=True)

print(f"Saving model to: {output_dir}")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("\n✅ Model saved successfully!")
print(f"\nTo load this model later:")
print(f"  tokenizer = AutoTokenizer.from_pretrained('{output_dir}')")
print(f"  model = AutoModelForSequenceClassification.from_pretrained('{output_dir}')")

## Test Prediction

Try the model on a sample text.

In [ ]:
def predict_quality(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    m = trainer.model.to(device)
    m.eval()

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=256
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = m(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)[0]
        predicted_class = int(torch.argmax(probs).item())
        confidence = float(probs[predicted_class].item())

    return {
        "prediction": "Good Answer" if predicted_class == 1 else "Bad Answer",
        "confidence": confidence,
        "class": predicted_class
    }


# Test on a validation example
if len(val_texts) > 0:
    test_text = val_texts[0]
    result = predict_quality(test_text)
    
    print("Test Prediction:")
    print(f"Text: {test_text[:100]}...")
    print(f"Prediction: {result['prediction']}")
    print(f"Confidence: {result['confidence']:.2%}")
    print(f"Actual label: {'Good' if val_labels[0] == 1 else 'Bad'}")